In [ ]:
pip install opencv-python

In [ ]:
pip install tensorflow

**IMPORTING LIBRARIES**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**CHECKING IMAGES FOR CONFIRMATION**

In [ ]:
plt.imshow(cv2.imread("../pothole/pothole-detection-dataset/potholes/125.jpg"))

In [ ]:
plt.imshow(cv2.imread("../pothole/pothole-detection-dataset/normal/11.jpg"))

**Part 1 - Data Preprocessing**


****Splitting dataset into training and validation and preprocessing the training set****

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)
training_set = train_datagen.flow_from_directory('../pothole/pothole-detection-dataset/',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary',
                                                 subset="training")

**Preprocessing the validation set**

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    "../pothole/pothole-detection-dataset", # same directory as training data
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation')

**Part 2 - Building the CNN**


**Initialising the CNN**

In [ ]:
cnn = tf.keras.models.Sequential()

**Step 1 - Convolution**


In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

**Step 2 - Pooling**


In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

**Adding a second convolutional layer**

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

**Step 3 - Flattening**


In [ ]:
cnn.add(tf.keras.layers.Flatten())


**Step 4 - Full Connection**


In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

**Step 5 - Output Layer**


In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

**Part 3 - Training the CNN**


**Compiling the CNN**

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

**Training the CNN on the Training set and evaluating it on the Validation set**

In [ ]:
cnn.fit(x = training_set, validation_data = validation_generator, epochs = 25)

**Part 4 - Making a single prediction**

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('../pothole/pothole-detection-dataset/trial2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'pothole'
else:
  prediction = 'normal'
  

In [ ]:
print(prediction)